### Assignment 7
#### Assignment 7.1
In this part of the assignment, you will partition a dataset using different strategies. You will use the routes.parquet dataset you created in a previous assignment. For this dataset, the key for each route will be the three-letter source airport code concatenated with the three-letter destination airport code and the two-letter airline. For instance, a route from Omaha Eppley Airfield (OMA) to Denver International Airport (DEN) on American Airlines (AA) has a key of OMADENAA.

a.
Start by loading the dataset from the previous assignment using Pandas's read_parquet method. Next, add the concatenated key then using Panda's apply method to create a new column called key. For this part of the example, we will create 16 partitions so that we can compare it to the partitions we create from hashed keys in the next part of the assignment. The partitions are determined by the first letter of the composite key using the following partitions.

In [2]:
import pandas as pd
import os
from pathlib import Path
import gzip
import hashlib
import shutil
import pygeohash
import s3fs
import json

In [2]:
path = '/home/jovyan/dsc650/dsc650/assignments/assignment03/results/routes.parquet'

In [16]:
parquet_pd = pd.read_parquet(path)

In [14]:
test_df = pd.DataFrame(parquet_pd)

In [15]:
test_df.head()

,airline,src_airport,dst_airport,codeshare,equipment
0,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2965.0, 'altitude': 89.0, 'city...","{'airport_id': 2990.0, 'altitude': 411.0, 'cit...",False,[CR2]
1,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2966.0, 'altitude': -65.0, 'cit...","{'airport_id': 2990.0, 'altitude': 411.0, 'cit...",False,[CR2]
2,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2966.0, 'altitude': -65.0, 'cit...","{'airport_id': 2962.0, 'altitude': 1054.0, 'ci...",False,[CR2]
3,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2968.0, 'altitude': 769.0, 'cit...","{'airport_id': 2990.0, 'altitude': 411.0, 'cit...",False,[CR2]
4,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2968.0, 'altitude': 769.0, 'cit...","{'airport_id': 4078.0, 'altitude': 365.0, 'cit...",False,[CR2]


In [3]:
#taken from week 3
endpoint_url='https://storage.budsc.midwest-datascience.com'
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)

In [4]:
#taken from week 3
def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
        

    return records

In [5]:
# parse out the json format 
def flatten_record(record):
    flat_record = dict()
    for key, value in record.items():
        if key in ['airline','src_airport','dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key= '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            flat_record[key] = value
    return flat_record

In [6]:
# call the parsed json function and put into a pd dataframe
def create_flattened_dataset():
    records = read_jsonl_data()
    parquest_path = results_dir.joinpath('routes-flattened.parquet')
    return pd.DataFrame.from_records([flatten_record(record) for record in records])

df = create_flattened_dataset()
#concatenate the data into a single key for each record
df['key'] = df['src_airport_iata'].astype(str)+df['dst_airport_iata'].astype(str)+df['airline_icao'].astype(str)

In [31]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_longitude,dst_airport_altitude,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZNARD
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZNARD
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,43.081902,1054.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRVARD
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZNARD
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,82.650703,365.0,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVBARD


In [7]:
def get_partition(key):
    partitions = (
        ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
        ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
        ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
        ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
    )
    # create the folders
    for start_partition, end_partition in partitions:
        if start_partition == end_partition:
            kv_key = start_partition
        else:
            kv_key = start_partition+'-'+end_partition
        if key[0] >= start_partition and key[0] <= end_partition:
            return kv_key

df['kv_key'] = df['key'].apply(get_partition)
#df.to_parquet()

In [36]:
# assign the folder names with the key values
df.to_parquet(results_dir, partition_cols=['kv_key'])

b.Next, we are going to partition the dataset again, but this time we will partition by the hash value of the key. The following is a function that will create a SHA256 hash of the input key and return a hexadecimal string representation of the hash.

In [8]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

In [9]:
#iterate over each row and assign the hashed key to each record
df['hashed'] = df.apply(lambda x: hash_key(x.key), axis=1)

In [10]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,hashed
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZNARD,A,aba8e675ed69dc08c83216c16223c87e7269e7a28a610d...
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZNARD,A,2b6ed0aa93298bb2ff79f07a4a7ca145ad8aab1fc62123...
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRVARD,A,749ae10a1b09952ee8c123e6b479054941e4aaa64b3edc...
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZNARD,C-D,14c51782ae565b75b98cdf6e2a44ee74751a7a39fd1536...
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVBARD,C-D,8cd2fe063f2bb82991dbc3655aefa321e7063b61f02a23...


In [10]:
df['hash_key'] = df['hashed'].str[:1]

In [12]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,hashed,hash_key
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZNARD,A,aba8e675ed69dc08c83216c16223c87e7269e7a28a610d...,a
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZNARD,A,2b6ed0aa93298bb2ff79f07a4a7ca145ad8aab1fc62123...,2
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRVARD,A,749ae10a1b09952ee8c123e6b479054941e4aaa64b3edc...,7
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZNARD,C-D,14c51782ae565b75b98cdf6e2a44ee74751a7a39fd1536...,1
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVBARD,C-D,8cd2fe063f2bb82991dbc3655aefa321e7063b61f02a23...,8


In [47]:
df.to_parquet(results_dir, partition_cols=['hash_key'])

c. Finally, we will simulate multiple geographically distributed data centers. For this example, we will assume we have three data centers located in the western, central, and eastern United States. Google lists the locations of their data centers and we will use the following locations for our three data centers.

In [20]:
df['src_airport_geohash'] = df.apply(
    lambda row: pygeohash.encode(row.src_airport_latitude, row.src_airport_longitude), axis=1
)

In [21]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,hashed,hash_key,src_airport_geohash,location
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,airport,OurAirports,False,[CR2],AERKZNARD,A,aba8e675ed69dc08c83216c16223c87e7269e7a28a610d...,a,szsrjjzd02b3,dqby34cjw922
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,airport,OurAirports,False,[CR2],ASFKZNARD,A,2b6ed0aa93298bb2ff79f07a4a7ca145ad8aab1fc62123...,2,v04pk3t5gbjj,dqby34cjw922
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,airport,OurAirports,False,[CR2],ASFMRVARD,A,749ae10a1b09952ee8c123e6b479054941e4aaa64b3edc...,7,v04pk3t5gbjj,dqby34cjw922
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,airport,OurAirports,False,[CR2],CEKKZNARD,C-D,14c51782ae565b75b98cdf6e2a44ee74751a7a39fd1536...,1,v3gdxs17du83,c21g6s0rs4c7
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,airport,OurAirports,False,[CR2],CEKOVBARD,C-D,8cd2fe063f2bb82991dbc3655aefa321e7063b61f02a23...,8,v3gdxs17du83,c21g6s0rs4c7


In [25]:
def determine_location(src_airport_geohash):
    locations = dict(
        central=pygeohash.encode(41.1544433, -96.0422378),
        ## TODO: add west and east
        west=pygeohash.encode(45.5945645, -121.1786823),
        east=pygeohash.encode(39.08344, -77.6497145)
    )
    #print(locations)
    distances = []#TODO: a list of centers and distances using the pygeohash.geohash_haversine_distance function
    # aided by Corrine from Teams
    for l, geohash in locations.items():
        haverstine = pygeohash.geohash_haversine_distance(src_airport_geohash,geohash)
        #print(haverstine)
        distances.append(tuple((haverstine,l)))
    
    distances.sort()
    return distances[0][1]

df['location'] = df['src_airport_geohash'].apply(determine_location)

In [26]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,hashed,hash_key,src_airport_geohash,location
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,airport,OurAirports,False,[CR2],AERKZNARD,A,aba8e675ed69dc08c83216c16223c87e7269e7a28a610d...,a,szsrjjzd02b3,east
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,airport,OurAirports,False,[CR2],ASFKZNARD,A,2b6ed0aa93298bb2ff79f07a4a7ca145ad8aab1fc62123...,2,v04pk3t5gbjj,east
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,airport,OurAirports,False,[CR2],ASFMRVARD,A,749ae10a1b09952ee8c123e6b479054941e4aaa64b3edc...,7,v04pk3t5gbjj,east
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,airport,OurAirports,False,[CR2],CEKKZNARD,C-D,14c51782ae565b75b98cdf6e2a44ee74751a7a39fd1536...,1,v3gdxs17du83,west
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,airport,OurAirports,False,[CR2],CEKOVBARD,C-D,8cd2fe063f2bb82991dbc3655aefa321e7063b61f02a23...,8,v3gdxs17du83,west


In [27]:
df.to_parquet('results/geo', partition_cols=['location'])

d. Create a Python function that takes as input a list of keys and the number of partitions and returns a list of keys sorted into the specified number of partitions. The partitions should be roughly equal in size. Furthermore, the partitions should have the property that each partition contains all the keys between the least key in the partition and the greatest key in the partition. In other words, the partitions should be ordered.

In [41]:
import numpy as np

In [109]:
keys = ['yield', 'cash', 'price', 'dollars', 'vgx', 'doge', 'eth', 'btc', 'interest', 'interest', 'interest', 'farm', 'stake', 'reward', 'bank', 'exchange']
names = set(keys)

In [110]:
def balance_partitions(keys, num_partitions, names):
    key_count = len(keys)
    #partition iterator
    p=1
    partitions  = []
    # round up
    key_round = np.ceil(key_count / num_partitions)
    for i,key in enumerate(keys):
        if i+1 > key_round*p:
            p+=1
        partitions.append(str(key)+ '-' + str(p))
    return partitions

In [111]:
balance_partitions(keys,5, names)

['yield-1',
 'cash-1',
 'price-1',
 'dollars-1',
 'vgx-2',
 'doge-2',
 'eth-2',
 'btc-2',
 'interest-3',
 'interest-3',
 'interest-3',
 'farm-3',
 'stake-4',
 'reward-4',
 'bank-4',
 'exchange-4']